In [2]:
import gym
from gym import spaces
import numpy as np
import pytesseract
import matplotlib.pyplot as plt
import cv2
import os
import imutils
import re
import pickle
import pyautogui
from PIL import ImageGrab, Image
from difflib import SequenceMatcher
import tensorflow as tf
from stable_baselines.common.env_checker import check_env
import time
import win32gui, win32con, win32com.client
import pyautogui

In [3]:
# need to include in script for pytesseract to work
pytesseract.pytesseract.tesseract_cmd = 'D:\\VideoGame_AI\\third_party\\Tesseract-OCR\\tesseract.exe'

In [4]:
 %matplotlib inline

## Create Custom Gym Environment for N Game

In [5]:
class CustomEnv(gym.Env):
    metadata={'render.modes':['human']}
    
    def __init__(self):
        super(CustomEnv, self).__init__()
        
        # bounds for image grab. Note: This is for when the game is fullscreen
        left_x_bound = 430
        top_y_bound = 170
        right_x_bound = 1485
        bottom_y_bound = 900
        self.whole_screen_bounds =(left_x_bound,top_y_bound,right_x_bound,bottom_y_bound)

        # bounds for amount of time left. Necessary for reward
        self.time_left_bounds = (490,170,565,190)

        # bounds for level completion message
        self.lvlComplete_msgBounds = (845,290,1070,310)

        # threshold for end level message detection
        self.lvlComplete_msgThresh = 0.80

        # shape of the observation
        self.observation_height = 530
        self.observation_width = 370
        self.observation_channels = 3
        
        
        # agent can go left,right, space
        self.action_space = spaces.Discrete(4)
        
        # input will be an image. for now we will say the shape is (60,60,3)
        self.observation_space = spaces.Box(low=0, high=255,
                                           shape=(self.observation_height,
                                                  self.observation_width,
                                                  self.observation_channels), 
                                            dtype=np.uint8)
        
    def reset(self):
        # press K to kill agent and reset
        pyautogui.press(action_lookup[3])
        
        # press SPACE to continue
        pyautogui.press(action_lookup[2])
        
        # press Z to start
        pyautogui.press(action_lookup[2])
        
        # capture the screen on restart
        observation, lvl_initialTime = self.screen_capture()
        
        # save so we can use for the reward function
        self.lvl_initialTime = lvl_initialTime
        
        self.prev_stepTime = self.lvl_initialTime
        
        return observation
    
    def step(self, action):
        action_key = action_lookup[action]
        
        # key press
        pyautogui.press(action_key)
        
        # capture updated screen
        observation, lvl_time = self.screen_capture()
        
        # determine if we are done. Capture end of level message and see how close
        self.text_ratio = self.detect_end_levelMessage()
        
        done = bool(self.text_ratio >= self.lvlComplete_msgThresh)

        # Two rewards here:
        # One for completing the level
        lvl_completeReward = 1 if done else 0
        
        # Another for the amount of time it took to complete the level (ratio)
        # If the agent has obtained a yellow square, which increases its current level time
        # then set the time reward equal to the ratio of current time to intial level time
        if lvl_time > self.prev_stepTime:
            time_rewardRatio = lvl_time/self.lvl_initialTime
        else:
            time_rewardRatio = 0
        
        self.prev_stepTime = lvl_time
        
        reward = time_rewardRatio+lvl_completeReward
        
        info = {}
        
        return observation, reward, done, info
    
    def screen_capture(self):
        
        # capture the screen on restart
        screen = np.array(ImageGrab.grab(bbox=self.whole_screen_bounds))
        
        # resize the screen
        observation = cv2.resize(screen,(self.observation_height,self.observation_width))
        
        # get the intial amount of time the agent has to complete the level
        time_capture = np.array(ImageGrab.grab(bbox=self.time_left_bounds))
        
        time_captureGS = cv2.cvtColor(time_capture,cv2.COLOR_BGR2GRAY)
        
        # use pytesseract to extract digits from image
        extracted_time = pytesseract.image_to_string(Image.fromarray(time_captureGS),lang='eng',
                           config='--psm 8 -c tessedit_char_whitelist=.0123456789',
                            output_type='string')
        
        # convert extracted time to float
        try:
            lvl_time = round(float(re.findall(r"[-+]?\d*\.\d+|\d+",extracted_time)[0]),2)
        except Exception:
            plt.imshow(time_captureGS)
            print(extracted_time)
            
        return observation, lvl_time
    
    def detect_end_levelMessage(self):
        
        # determine if the end level message is present
        message_capture = np.array(ImageGrab.grab(bbox=self.lvlComplete_msgBounds))
        
        message_gs = cv2.cvtColor(message_capture,cv2.COLOR_BGR2GRAY)
        
        extracted_text = pytesseract.image_to_string(Image.fromarray(message_gs),
                            config = '--psm 7',
                            lang='eng')
        
        message_text = extracted_text.rstrip()
        
        # determine how similar the text is to the true message
        true_message = 'level complete! press JUMP to continue'
        
        # get similarity between true text and extracted text
        textSimilarity_ratio = round(SequenceMatcher(None,message_text,true_message).ratio(),2)
        
        return textSimilarity_ratio
    
    def close(self):
        pass
        
# action dictionary. Note: 0x4B is mapped onto the 'K' key
action_lookup = {
    0 : 'left',
    1 : 'right',
    2 : 'z',
    3 : 'k'
}     

Check if Environment works

In [6]:
env = CustomEnv()

In [331]:
check_env(env,warn=True)

In [117]:
# start the game
os.startfile('D:\\Nv2-PC.exe')
first_game_start = 1

# TO DO:

- [ ] Figure out how to send consistent key inputs to game
- [ ] Create NN Agent

In [171]:
# switch focus to the game window so actions register
hwnd = win32gui.FindWindow(None,'Adobe Flash Player 11')

# do this to avoid errors
shell = win32com.client.Dispatch("WScript.Shell")
shell.SendKeys('%')

# set focus on window
win32gui.SetForegroundWindow(hwnd)

# maximize window
tup = win32gui.GetWindowPlacement(hwnd)
if tup[1] != win32con.SW_SHOWMAXIMIZED:
    win32gui.ShowWindow(hwnd, win32con.SW_MAXIMIZE)

    
if first_game_start:
    # click on play game and first level
    play_game_xy = (1244,427)

    select_lvl_xy = (549,323)

    pyautogui.click(x = play_game_xy[0], y = play_game_xy[1])
    pyautogui.click(x = select_lvl_xy[0], y = select_lvl_xy[1])

    first_game_start = 0

time.sleep(3)
env = CustomEnv()
observation = env.reset()
for x in range(0,5):
    obs, rew, done, info = env.step(env.action_space.sample())